In [3]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [51]:
import json
from helpers.chat import get_openai_completion, make_function_call, get_prompt, make_structured_call
from helpers.functions import parse_type
from IPython.display import Markdown

In [6]:
import openai
from openai import OpenAI
from helpers.functions import create_pydantic_model, get_fields

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [9]:
stage_name = 'profile'

In [211]:
extraction_data   = get_fields(f"{stage_name}/extract")

In [117]:
probe_fields      = get_fields(f"{stage_name}/probe")

In [193]:
example_text = "Here are some examples of how you should probe the learner for information, and how you should respond once you get an answer.\n\n"

for f in probe_fields:
    name = f['field']
    q    = f['question']
    r    = f['response_template']
    ex_string = f"""
    **{name}**:
    
    Question: {q}
    Response: {r}
    """
    example_text += ex_string

In [226]:
missing_model = create_pydantic_model(
    f"identify_{stage_name}_attributes",
    fields=extraction_data['filter_fields'],
    description="List down all the attributes we can extract from the user's current response."
)

In [227]:
field_summary = ''
for field in extraction_data['required_fields']:
    field_text = f"- **{field['title']}**\n[{field['annotation']}]\n{field['description']}\n\n"
    field_summary += field_text

In [228]:
extract_system_template = get_prompt(f"{stage_name}/extract").format(FIELDS=field_summary)
extract_system_message  = {"role":"system","content":extract_system_template}

In [261]:
user_content = """
Zavmo: What new information can we extract from the user's response?
Learner: I just finished my bachelors
"""
user_message = {"role":"user","content":user_content}

In [262]:
missing_tools = force_tool_call(tools=[missing_model],
                              messages=[extract_system_message,user_message])

In [263]:
missing_tools[0].attributes

['edu_level', 'current_role']

In [264]:
required_attributes = extraction_data['required_fields']

In [265]:
new_attributes = missing_tools[0].attributes

In [266]:
model = create_pydantic_model(f"{stage_name}_new_attributes", 
                              fields=[f for f in extraction_data['required_fields'] if f['title'] in new_attributes], 
                              description="New information that we learned about a learner."
                             )

In [267]:
extracted_data = force_tool_call(tools=[model],
                              messages=[extract_system_message,user_message])

In [268]:
extracted_data[0].model_dump()

{'edu_level': 5, 'current_role': 'Graduate'}

In [49]:
response = openai_client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[probe_system_msg],
    #tools=tools,
    #tool_choice="required",
    #parallel_tool_calls=True,
)

In [50]:
Markdown(response.choices[0].message.content)

Hey there! 🎉 I'm so excited to start this learning adventure with you! Before we dive in, I'd love to get to know you better. This will help me tailor our experience to fit your interests and goals. 

Could you share your name, what you're looking to learn about, and any preferences you have for our sessions? For example, do you prefer more visual explanations, hands-on activities, or straightforward discussions? 🌟 

In [ ]:
tool_calls = response.choices[0].message.tool_calls

In [ ]:
model = model.parse_raw(tool_calls[0].function.arguments)

In [ ]:
# Extractor Bot Logic
class ExtractorBot:
    def __init__(self):
        self.profile_data = {}
        self.required_fields = ["first_name", "last_name", "current_role", "learning_interests"]
        self.completed_fields = []
        self.tools = [openai.pydantic_function_tool(i) for i in [GetFirstName, GetLastName, GetCurrentRole, GetLearningInterests]]

    def extract_data(self, messages):
        # Prepare and send messages for tool calls
        tool_sys_msg = get_prompt('extractor.md', 'assets/prompts/profile').format(
            required_fields=", ".join(self.required_fields) if self.required_fields else "No pending fields to extract"
        )
        tool_messages = [{"role": "system", "content": tool_sys_msg}] + messages[1:]
        tool_calls = make_tool_calls(tool_messages, self.tools)

        if tool_calls:
            for tool_response in tool_calls:
                arg = tool_response.function.arguments
                self.profile_data.update(json.loads(arg))

            # Update fields status
            self.completed_fields = list(set(self.profile_data))
            self.required_fields = [field for field in self.required_fields if field not in self.completed_fields]

    def are_all_fields_collected(self):
        return not self.required_fields

    def get_confirmation(self, messages):
        answer = make_structured_call(GetConfirmation, messages).answer
        return 'Yes' if answer and answer.value == 'Yes' else 'No'


# Guiding Bot Logic
class ProfileCollector:
    def __init__(self, extractor_bot):
        self.extractor_bot = extractor_bot
        self.history = []
        self.confirmed = "No"

    def system_message(self):
        prompt = get_prompt("system.md", "assets/prompts/profile")
        if self.extractor_bot.are_all_fields_collected():
            prompt += "\nAsk for confirmation (Yes/No) sharing details" + f"\n\n```json\n{json.dumps(self.extractor_bot.profile_data, indent=4)}\n```"
        return {"role": "system", "content": prompt}

    def user_message(self, input=""):
        # Generate user message with current field statuses
        completed = ", ".join(self.extractor_bot.completed_fields) or "Not completed any field yet."
        required = ", ".join(self.extractor_bot.required_fields) or "Received information for all remaining fields."
        content = f"Field Extraction Information\n\nCompleted Fields:\n{completed}\n\nRequired Fields:\n{required}\n\nUser response:\n{input}"
        self.user_mssg = {"role": "user", "content": content}

    def ask_question(self):
        # Check if all fields are collected and ask for confirmation if needed
        try: 
            self.user_mssg
        except:
            self.user_message()

        if self.extractor_bot.are_all_fields_collected():
            user_mssg = self.user_mssg 
            user_mssg['content'] = user_mssg['content'] + "\n\nIf updates on any field is required and user will mention corresponding to what is asked, or if user says No. Take it as a No"
            messages = [self.system_message()] + self.history + [user_mssg]
            confirmation = self.extractor_bot.get_confirmation(messages)
            
            if confirmation == "Yes":
                self.confirmed = "Yes"
                return self.summarize_profile()

        # Collect remaining fields or reattempt extraction if confirmation is "No"
        if not self.extractor_bot.are_all_fields_collected() or self.confirmed != "Yes":
            tool_messages = self.history + [self.user_mssg]
            self.extractor_bot.extract_data(tool_messages)

            # Generate and return assistant message
            messages = [self.system_message()] + self.history + [self.user_mssg]
            assistant_message = get_openai_completion(messages)
            messages.append({"role": "assistant", "content": assistant_message})
            self.history = messages[1:]  # Update history
            return assistant_message

    def summarize_profile(self):
        return f"Great! Your profile is complete:\n\n```json\n{json.dumps(self.extractor_bot.profile_data, indent=4)}\n```"


In [ ]:
extractor_bot = ExtractorBot()
pc            = ProfileCollector(extractor_bot)

In [ ]:
Markdown(pc.ask_question())

In [ ]:
pc.user_message("Yes")

In [ ]:
pc.user_message("Mumtaz Rahmani, current role is Junior software engineer, learning interests are invloved around LLMs and stats")

In [ ]:
pc.extractor_bot.profile_data

In [ ]:
messages = pc.history

In [ ]:
tool_sys_mssg = get_prompt('extractor.md','assets/prompts/profile').format(required_fields=", ".join(pc.extractor_bot.required_fields) if pc.extractor_bot.required_fields else "No pendig fields to extract")
tool_messages = [{"role":"system","content":tool_sys_mssg}]
tool_messages.extend(messages) 

In [ ]:
messages.append({"role":"user","content":"Yes"})

In [ ]:
answer = make_structured_call(GetConfirmation,messages).answer
if answer:
    print(answer.value)

## Internally sending tool response to model 

In [ ]:
profile_data={}

In [ ]:
message = get_openai_completion(messages, tools=tools, tool_choice="auto", parallel_tool_calls=True)

tool_calls = message.tool_calls
if tool_calls:
    print(f"Tools called: {[tool.function.name for tool in tool_calls]}")
    function_call_messages=[message]
    for tool in tool_calls:
        arguments = tool.function.arguments
        function_call_messages.extend([{"role": "tool", "content": json.dumps(arguments), "tool_call_id": tool.id}])
        profile_data.update(json.loads(arguments))
    messages.extend(function_call_messages)
    while message.content==None:
        message=get_openai_completion(messages, tools=tools, tool_choice="auto", parallel_tool_calls=True)
        print("Running in while loop")

assistant_response = message.content

In [ ]:
Markdown(assistant_response)

In [ ]:
messages.append({"role":"assistant","content":assistant_response})

In [ ]:
profile_data

In [ ]:
def get_claude_response(sys_mssg,messages,model="claude-3-opus-20240229"):
    client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
    message = client.messages.create(
        model=model,
        max_tokens=1024,
        system=sys_mssg,
        messages=messages
    )
    return message.content[0].text

claude_response = get_claude_response(sys_mssg,messages[1:])
claude_response